In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import mediapipe as mp
from time import process_time
import os

C:\Users\mahaj\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
os.environ["GLOG_minloglevel"] ="2"

In [3]:
dict ={
    0: "WRIST",
    1: "THUMB_CMC",
    2: "THUMB_MCP",
    3: "THUMB_IP",
    4: "THUMB_TIP",
    5: "INDEX_FINGER_MCP",
    6: "INDEX_FINGER_PIP",
    7: "INDEX_FINGER_DIP",
    8: "INDEX_FINGER_TIP",
    9:  "MIDDLE_FINGER_MCP",
    10: "MIDDLE_FINGER_PIP",
    11: "MIDDLE_FINGER_DIP",
    12: "MIDDLE_FINGER_TIP",
    13: "RING_FINGER_MCP",
    14: "RING_FINGER_PIP",
    15: "RING_FINGER_DIP",
    16: "RING_FINGER_TIP",
    17: "PINKY_MCP",
    18: "PINKY_PIP",
    19: "PINKY_DIP",
    20: "PINKY_TIP",
}
mp_hands = mp.solutions.hands
hands = mp_hands.Hands( 
    static_image_mode= True,
    max_num_hands = 3,
    min_detection_confidence=0.5)

In [38]:
## Transform the image to a vector with 21 coordinates.
def image_to_vector(img):
    try:
        result_vector =[]
        result = hands.process(img)

        for hand_landmark in result.multi_hand_world_landmarks:
            for item in dict:
                landmarkObj = hand_landmark.landmark[mp_hands.HandLandmark[dict[item]]]
                result_vector.append([landmarkObj.x,landmarkObj.y, landmarkObj.z])
        
        return np.array(result_vector, dtype=np.float64)
    
    except:
        return np.reshape(np.zeros(63, dtype=np.float64), (21,3))

In [39]:
def NormalizeData(data):
    for i in range(len(data)):
        data[i] = (data[i] - np.min(data[i])) / (np.max(data[i]) - np.min(data[i]))
    return data

In [6]:
## Load dataset
(train_images,train_labels) = tfds.as_numpy(tfds.load('rock_paper_scissors', split='train[:70%]', batch_size=-1, as_supervised = True))
(validation_images,validation_labels) = tfds.as_numpy(tfds.load('rock_paper_scissors', split='train[70%:]', batch_size=-1, as_supervised = True))
(test_images,test_labels) = tfds.as_numpy(tfds.load('rock_paper_scissors', split='test', batch_size=-1, as_supervised = True))

In [44]:
#Transform the images to vectors
train_map = map(image_to_vector, train_images)
test_map = map(image_to_vector, test_images)
validation_map = map(image_to_vector, validation_images)

In [45]:
raw_train_data = np.array(list(train_map))
raw_test_data = np.array(list(test_map))
raw_validation_data = np.array(list(validation_map))

In [53]:
raw_train_data = np.reshape(raw_train_data, (-1, 63))
raw_test_data = np.reshape(raw_test_data, (-1, 63))
raw_validation_data = np.reshape(raw_validation_data, (-1, 63))

In [61]:
#Check where sum of a row is not 0
train_data, train_labels =  raw_train_data[np.where(np.sum(raw_train_data, axis=1) != 0)], train_labels[np.where(np.sum(raw_train_data, axis=1) != 0)]
test_data, test_labels =  raw_test_data[np.where(np.sum(raw_test_data, axis=1) != 0)], test_labels[np.where(np.sum(raw_test_data, axis=1) != 0)]
validation_data, validation_labels =  raw_validation_data[np.where(np.sum(raw_validation_data, axis=1) != 0)], validation_labels[np.where(np.sum(raw_validation_data, axis=1) != 0)]

In [62]:
train_normalized = NormalizeData(train_data)
test_normalized = NormalizeData(test_data)
validation_normalized = NormalizeData(validation_data)

In [66]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss') < 0.01):
            print("\nReached 1% loss so cancelling training!")
            self.model.stop_training = True

In [67]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(63,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')])

In [68]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [71]:
model.fit(train_normalized, train_labels, epochs=20, batch_size=10, callbacks=[myCallback()], validation_data=(validation_normalized, validation_labels))

Epoch 1/20
169/169 [==============================] - 1s 5ms/step - loss: 0.0272 - accuracy: 0.9929 - val_loss: 0.0128 - val_accuracy: 0.9973
Epoch 2/20
169/169 [==============================] - 1s 5ms/step - loss: 0.0326 - accuracy: 0.9905 - val_loss: 0.0175 - val_accuracy: 0.9973
Epoch 3/20
169/169 [==============================] - 1s 5ms/step - loss: 0.0251 - accuracy: 0.9929 - val_loss: 0.0215 - val_accuracy: 0.9959
Epoch 4/20
169/169 [==============================] - 1s 5ms/step - loss: 0.0447 - accuracy: 0.9929 - val_loss: 0.0145 - val_accuracy: 0.9973
Epoch 5/20
169/169 [==============================] - 1s 5ms/step - loss: 0.0234 - accuracy: 0.9952 - val_loss: 0.0159 - val_accuracy: 0.9973
Epoch 6/20
169/169 [==============================] - 1s 5ms/step - loss: 0.0274 - accuracy: 0.9935 - val_loss: 0.0119 - val_accuracy: 0.9986
Epoch 7/20
169/169 [==============================] - 1s 5ms/step - loss: 0.0241 - accuracy: 0.9923 - val_loss: 0.0600 - val_accuracy: 0.9767
Epoch 

In [93]:
model.evaluate(test_normalized, test_labels)

369/369 [==============================] - 1s 3ms/step - loss: 0.1097 - accuracy: 0.9729


[0.1097118929028511, 0.9728997349739075]

In [92]:
model.save_weights('model_weights.h5')
model.save('model.h5')